# Sheet for extracting raw-data from Porto dataset

In [1]:
# Importing necessary modules

import pandas as pd
import os, shutil


# From utils
from utils.alphabetical_number import increment_alphabetical
from utils.trajectory_distance import calculate_trajectory_distance
from utils.metafile_handler import create_meta_files, get_meta_files, delete_meta_files


In [2]:
# Containning variables and constants for this spreadsheet
SHOULD_DELETE_OLD_FILES= True

OUTPUT_FOLDER = "../data/chosen_data/porto/"
RAW_DATA_FILE = "../data/raw_data/train.csv"

LOG = False # Set to true for printing during data extraction

NUMBER_OF_TRACES = 1000

In [3]:
# Calculating the size of the minimum bounding rectangle

X = calculate_trajectory_distance([ ( MIN_LAT, MIN_LON ) , ( MAX_LAT, MIN_LON ) ])
Y = calculate_trajectory_distance([ ( MIN_LAT, MIN_LON ) , ( MIN_LAT, MAX_LON ) ])

print(f"Bounded rectangle have sidelengths (X,Y): {X, Y} km")

Bounded rectangle have sidelengths (X,Y): (5.559754011677007, 7.536733782089804) km


In [4]:
# Reading the dataset into dataframe

raw_df = pd.read_csv(RAW_DATA_FILE)

FileNotFoundError: [Errno 2] No such file or directory: '../data/raw_data/train.csv'

In [ ]:
# Run this cell to clear the chosen files in the PORTO folder

if SHOULD_DELETE_OLD_FILES:
    for filename in os.listdir(OUTPUT_FOLDER):
        file_path = os.path.join(OUTPUT_FOLDER, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print("Failed to remove %s. Reason: %s" % (file_path, e))

In [ ]:
# This cell will read the data and generate a file for each trajectory in the given output directory
# Will choose traces longer than n points that are within a given rectangle of the city

counter = 0
name_counter = "AAA"

for index, row in raw_df.iterrows():
    trace_id = row["TRIP_ID"] 

    # If row is misssing data: ignore row
    if row["MISSING_DATA"] == True: 
        if LOG: print(trace_id, "is missing data") 
        continue

    trace = row["POLYLINE"][2:-2].split("],[")

    # If trace-length less than "MIN_LEN": ignore row
    if len(trace) < MIN_LEN: 
        if LOG:  print(trace_id, "is less than preferred length")
        continue


    # If trace are outside bounded rectangle or noisy data: ignore row
    
    last_coordinate = trace[0]
    for coordinate in trace:

        lon, lat = list(map(float, coordinate.split(",")))

        # Outside bounded rectangle
        if ( not ( MIN_LAT <= lat <= MAX_LAT )) or ( not ( MIN_LON <= lon <= MAX_LON )):
            if LOG: print(trace_id, "is outside bounded rectangle")
            break

        # Traces with noisy data:
        if last_coordinate != coordinate:
            last_lon, last_lat = list(map(float, last_coordinate.split(",")))
            distance = calculate_trajectory_distance([(last_lat, last_lon), (lat, lon)])
            
            # If distance between two consecutive coordinates are too great:
            if distance > MAX_DIST_BETWEEN_COORDINATES:
                if LOG: print("Possibly noisy data - continuing with next trace")
                break 
            

        last_coordinate = coordinate
        # IMPLEMENT HERE, use distance function 

    # Else, everything is good so far --> Write trajectory to file
    else:        
        with open(f'{OUTPUT_FOLDER}/P_{name_counter}.txt','w') as file:
            for coordinate in trace:
                lon, lat = coordinate.split(",")
                file.write("%s, %s\n" % (lat, lon))
            file.close()

        counter +=1
        
        name_counter = increment_alphabetical(name_counter)

        if counter >= NUMBER_OF_TRACES: 
            break

print("Number of trajectories written to file:", counter)

Number of trajectories written to file: 1000


In [ ]:
# Creating the metafiles that will function as index files for the datasets

if get_meta_files(OUTPUT_FOLDER):
    delete_meta_files(OUTPUT_FOLDER)

create_meta_files(path_to_files=OUTPUT_FOLDER, data_prefix="P_", create_test_set=True)